In [1]:
import pandas as pd
import requests
import json

In [2]:
g_perth_sa2 = pd.read_csv("SA2_2016_AUST.csv")
g_perth_sa2.head()

,SA2_MAINCODE_2016,SA2_5DIGITCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,GCCSA_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099


In [3]:
g_perth_sa2 = g_perth_sa2.loc[g_perth_sa2["GCCSA_NAME_2016"] == "Greater Perth"]
g_perth_cols = ["SA2_MAINCODE_2016", "SA2_NAME_2016","GCCSA_NAME_2016"]
g_perth_sa2= g_perth_sa2[g_perth_cols].copy()
g_perth_sa2.dtypes
g_perth_sa2["SA2_MAINCODE_2016"] = g_perth_sa2["SA2_MAINCODE_2016"].apply(str)

In [4]:
#make a list of SA2 Maincode for ABS API
sa2_list = g_perth_sa2["SA2_MAINCODE_2016"].tolist()
household_dict = {"SA2 Code" : [], "SA2 Name":[], "Household Size":[], "2006":[], "2011":[], "2016":[]}

In [5]:
# Rent 2006 2011 2016
household_url = "http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T23_TS_SA/TOT.1.5.SA2.507041178/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2006&endPeriod=2016"

bins = ['TOT','1','2','3','4', '5', '6', '7', '8']
    
for sa2 in sa2_list:
    for i in bins:
             response = requests.get(f"http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T23_TS_SA/TOT.{i}.5.SA2.{sa2}/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2006&endPeriod=2016")
             household_size = response.json()
#              print(household_size)
             
             household_dict["SA2 Code"].append(household_size["structure"]["dimensions"]["observation"][4]["values"][0]["id"])
             household_dict["SA2 Name"].append(household_size["structure"]["dimensions"]["observation"][4]["values"][0]["name"])
             household_dict["Household Size"].append(household_size["structure"]["dimensions"]["observation"][1]["values"][0]["name"])
             household_dict["2006"].append(household_size["dataSets"][0]["observations"]["0:0:0:0:0:0"][0])
             household_dict["2011"].append(household_size["dataSets"][0]["observations"]["0:0:0:0:0:1"][0])
             household_dict["2016"].append(household_size["dataSets"][0]["observations"]["0:0:0:0:0:2"][0])

In [6]:
household_df = pd.DataFrame(household_dict)
print(household_df)

       SA2 Code              SA2 Name         Household Size    2006    2011  \
0     502011021  Dawesville - Bouvard                  Total  1389.0  1970.0   
1     502011021  Dawesville - Bouvard             One person   211.0   355.0   
2     502011021  Dawesville - Bouvard            Two persons   610.0   817.0   
3     502011021  Dawesville - Bouvard          Three persons   184.0   267.0   
4     502011021  Dawesville - Bouvard           Four persons   240.0   350.0   
...         ...                   ...                    ...     ...     ...   
1552  507051193               Warnbro           Four persons   652.0   705.0   
1553  507051193               Warnbro           Five persons   320.0   289.0   
1554  507051193               Warnbro            Six persons   131.0   122.0   
1555  507051193               Warnbro          Seven persons    30.0    20.0   
1556  507051193               Warnbro  Eight or more persons    15.0    18.0   

        2016  
0     2515.0  
1      45

In [8]:
household_df.to_csv('Total Household 2006-2016.csv')